In [ ]:
# !pip install psycopg2

In [1]:
import psycopg2 as ps
import json

In [2]:
try:
    with open('hexchat_db_key.json', 'r') as f:
        json_new_obj = f.read()
        credentials = json.loads(json_new_obj)
except:
    print("Get hexchat_db_key.json if you do not have") # from Nishat 

Get hexchat_db_key.json if you do not have


In [29]:
latest_form_group_json = {
    "metadata": {
        "group_name": "test_group",
        "group_status": "ALIVE",  # ALIVE / DEAD, update by Frontend if current time pass end_time, then show them connect and questionnarie things
        "start_time": 1587848400,
        "end_time": 1588021200,  # check against this timestamp to determine whether the room is ended
        "members": [
            "KkOoBVwjT3ZEmuNOtRI5yeMujgr1",
            "Rxlb9gYYdLep6Ts810T18eq9Ijm2",
            "4UOjpg8j4iNgR3JB50cSPqqRKj73",
        ],
    },
    "messages": [],  # chat messages will go here
    "messages_cnt": 0,  # count message sum
    # we should have 6 for this, 3 is for example only
    "connection": {  # after chat, if user choose to connect with others, add other uids in array, then check if their uid exist in other lists
        "KkOoBVwjT3ZEmuNOtRI5yeMujgr1": [],
        "Rxlb9gYYdLep6Ts810T18eq9Ijm2": [],
        "4UOjpg8j4iNgR3JB50cSPqqRKj73": [],
    },
    # we should have 6 for this, 3 is for example only
    # hardcode the questions for the moment if we don't have time
    "questionnaire": {
        "KkOoBVwjT3ZEmuNOtRI5yeMujgr1": {},
        "Rxlb9gYYdLep6Ts810T18eq9Ijm2": {},
        "4UOjpg8j4iNgR3JB50cSPqqRKj73": {},
    },
}

In [73]:
group_json = json.dumps(latest_form_group_json)

In [42]:
member_list = latest_form_group_json['metadata']['members']
member_list

['KkOoBVwjT3ZEmuNOtRI5yeMujgr1',
 'Rxlb9gYYdLep6Ts810T18eq9Ijm2',
 '4UOjpg8j4iNgR3JB50cSPqqRKj73']

In [46]:
date_list = [str(latest_form_group_json['metadata']['start_time'])]
date_list

['1587848400']

In [84]:
uniq_id = str(hash(tuple(member_list+date_list)))
type(uniq_id)

str

In [82]:
# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])
cur = conn.cursor()

In [83]:
create_table = """
    CREATE TABLE IF NOT EXISTS groups (
    ID TEXT unique,
    formed_group json NOT NULL
);
    """

# Execute SQL Command and commit to DB
cur.execute(create_table)
conn.commit()

In [87]:
table_name = 'groups'
sql_string = f'INSERT INTO {table_name} '
# sql_string += "(" + ' VALUES '.join(latest_form_group_json) + ")\n "
sql_string +='(ID, formed_group)' + f" VALUES ('{uniq_id}', '{group_json}')"

In [88]:
sql_string

'INSERT INTO groups (ID, formed_group) VALUES (\'8758000244026113573\', \'{"metadata": {"group_name": "test_group", "group_status": "ALIVE", "start_time": 1587848400, "end_time": 1588021200, "members": ["KkOoBVwjT3ZEmuNOtRI5yeMujgr1", "Rxlb9gYYdLep6Ts810T18eq9Ijm2", "4UOjpg8j4iNgR3JB50cSPqqRKj73"]}, "messages": [], "messages_cnt": 0, "connection": {"KkOoBVwjT3ZEmuNOtRI5yeMujgr1": [], "Rxlb9gYYdLep6Ts810T18eq9Ijm2": [], "4UOjpg8j4iNgR3JB50cSPqqRKj73": []}, "questionnaire": {"KkOoBVwjT3ZEmuNOtRI5yeMujgr1": {}, "Rxlb9gYYdLep6Ts810T18eq9Ijm2": {}, "4UOjpg8j4iNgR3JB50cSPqqRKj73": {}}}\')'

In [89]:
# Execute SQL Command and commit to DB
cur.execute(sql_string)
conn.commit()